# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read csv to a dataframe
df = pd.read_csv("output/city_data.csv")


In [3]:
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cape Town,-33.93,18.42,14.00,76,75,8.20,ZA,1598623732
1,1,Beipiao,41.79,120.78,16.84,88,23,3.37,CN,1598623853
2,2,Fukuechō,32.69,128.84,27.84,85,1,6.17,JP,1598623853
3,3,Tiksi,71.69,128.87,6.16,78,48,2.45,RU,1598623854
4,4,São Filipe,14.90,-24.50,26.11,82,91,2.55,CV,1598623854


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations = df[["Lat", "Lng"]]
weight = df["Humidity"]

In [6]:
#add a map and add heatmap layer to the map
city_data_fig = gmaps.figure()


In [7]:
#heat layer
city_heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating = False, 
                                     max_intensity = 100, point_radius = 1)

In [8]:
city_data_fig.add_layer(city_heat_layer)

In [9]:
city_data_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
#ideal vacation city, max temp of 26, min temp of 19
df = df.loc[(df["Max Temp"] >= 19) & (df["Max Temp"] <=26)]

In [11]:
#humidity between 30 to 50 percent
df = df.loc[(df["Humidity"] >= 30) & (df["Humidity"] <=50)]

In [12]:
#windspeed between 2 to 5mph
df = df.loc[(df["Wind Speed"] >= 0) & (df["Wind Speed"] <=5)]

In [13]:
#cloudiness under 10
df = df.loc[df["Cloudiness"] <= 35]

In [14]:
#check number of vacation spots passing all above filters
df.count()

Unnamed: 0    9
City          9
Lat           9
Lng           9
Max Temp      9
Humidity      9
Cloudiness    9
Wind Speed    9
Country       9
Date          9
dtype: int64

In [15]:
#check dataframe
df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,Lanzhou,36.06,103.79,24.44,43,18,1.34,CN,1598623885
91,91,Castro,-24.79,-50.01,22.95,41,0,1.77,BR,1598623910
208,208,Antas,41.74,-8.31,22.78,45,0,1.79,PT,1598623981
233,233,Salta,-24.79,-65.41,19.00,45,0,3.10,AR,1598623996
371,371,Lichinga,-13.31,35.24,26.00,34,20,4.10,MZ,1598624075
474,474,Kushmurun,52.45,64.63,25.61,47,6,3.33,KZ,1598624137
489,489,Derzhavīnsk,51.10,66.32,24.65,36,17,1.86,KZ,1598624146
511,511,Atasū,48.68,71.64,21.13,39,0,3.40,KZ,1598624159
515,515,Makungu,-8.73,35.28,24.68,39,0,4.24,TZ,1598624162


In [16]:
#drop Unnamed column

df = df.drop(["Unnamed: 0"], axis = 1)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
#Store into variable named hotel_df
hotel_df = df.copy()


In [18]:
#reset df index to loop through the df usnig index for google API calls
hotel_df.reset_index()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,51,Lanzhou,36.06,103.79,24.44,43,18,1.34,CN,1598623885
1,91,Castro,-24.79,-50.01,22.95,41,0,1.77,BR,1598623910
2,208,Antas,41.74,-8.31,22.78,45,0,1.79,PT,1598623981
3,233,Salta,-24.79,-65.41,19.00,45,0,3.10,AR,1598623996
4,371,Lichinga,-13.31,35.24,26.00,34,20,4.10,MZ,1598624075
5,474,Kushmurun,52.45,64.63,25.61,47,6,3.33,KZ,1598624137
6,489,Derzhavīnsk,51.10,66.32,24.65,36,17,1.86,KZ,1598624146
7,511,Atasū,48.68,71.64,21.13,39,0,3.40,KZ,1598624159
8,515,Makungu,-8.73,35.28,24.68,39,0,4.24,TZ,1598624162


In [19]:
#Set parameters to search for hotels with 5000 meters.
search_radius = 5000
search_type = "lodging"

In [20]:
#create empty list to store hotel info
hotel = []

In [21]:
#run a loop to find the nearest hotel based on google maps API
for index, row in hotel_df.iterrows():
    search_coordinates = f"{row['Lat']},{row['Lng']}"
    
    #create a dictionary to hold parameters
    parameters = {
        "location": search_coordinates,
        "radius": search_radius,
        'type': search_type,
        "key":g_key
    }
    
    #set base_url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params = parameters)
    response = request.json()
    
    #if hotel info is retireved add it to hotel list else move to next
    try:
        hotel.append(response["results"][0]["name"])
    except:
        hotel.append("No hotels found within 5km radius")

In [22]:
#check hotels
hotel

['Lanzhou Amdo Xiangbala Hotel',
 'CHACARA BAILLY',
 'Rio Homem',
 'Hotel Alejandro I',
 'Residencial Bendiak',
 'Gostinitsa Pri Depo',
 'Ooo.khim Baza',
 'Dostyk',
 'Complex']

In [23]:
#add hotel to dataframe
hotel_df["Hotels"] = hotel

In [24]:
#verify new column is present in dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotels
51,Lanzhou,36.06,103.79,24.44,43,18,1.34,CN,1598623885,Lanzhou Amdo Xiangbala Hotel
91,Castro,-24.79,-50.01,22.95,41,0,1.77,BR,1598623910,CHACARA BAILLY
208,Antas,41.74,-8.31,22.78,45,0,1.79,PT,1598623981,Rio Homem
233,Salta,-24.79,-65.41,19.00,45,0,3.10,AR,1598623996,Hotel Alejandro I
371,Lichinga,-13.31,35.24,26.00,34,20,4.10,MZ,1598624075,Residencial Bendiak
474,Kushmurun,52.45,64.63,25.61,47,6,3.33,KZ,1598624137,Gostinitsa Pri Depo
489,Derzhavīnsk,51.10,66.32,24.65,36,17,1.86,KZ,1598624146,Ooo.khim Baza
511,Atasū,48.68,71.64,21.13,39,0,3.40,KZ,1598624159,Dostyk
515,Makungu,-8.73,35.28,24.68,39,0,4.24,TZ,1598624162,Complex


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotels}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_1 = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
figure = gmaps.figure()
heat_map = gmaps.heatmap_layer(locations, weights = weight,
                                dissipating = False, max_intensity = 100,
                                point_radius = 1)

marker_layer = gmaps.marker_layer(locations_1)
figure.add_layer(heat_map)
figure.add_layer(marker_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))